# ABOUT
__Author__: Pat McCornack

__Date__: 5/29/2024

__Purpose:__ 

----

In [12]:
import os
import glob

import numpy as np
import pandas as pd

import rasterio
from rasterio.merge import merge
from rasterio.windows import Window

import datetime as dt

from joblib import load
from joblib import Parallel, delayed

from IPython.display import clear_output


In [13]:
# Define filepaths
local_root_dir = r"C:\Users\mcco573\OneDrive - PNNL\Documents\_Projects\BPA Wildfire\fuelscape_modeling" 
pnnl_root_dir = r"\\pnl\projects\BPAWildfire\data\Landfire\fuels_modeling\fuelscape_modeling"

# Choose whether to work from local or drive
active_root_dir = pnnl_root_dir


# Define the source raster file names
data_dir =  os.path.join(active_root_dir, r'..\LF_raster_data\bpa_service_territory')
ref_data_dir = os.path.join(data_dir, r"..\_tables")
raster_fpaths = {
    "LF20_F40" : os.path.join(data_dir, "LC22_F40_220_bpa.tif"),
    "LF20_FVT" : os.path.join(data_dir, "LC22_FVT_220_bpa.tif"),
    "LF22_FVT" : os.path.join(data_dir, "LC22_FVT_230_bpa.tif"),
    "LF20_FVC" : os.path.join(data_dir, "LC22_FVC_220_bpa.tif"),
    "LF22_FVC" : os.path.join(data_dir, "LC22_FVC_230_bpa.tif"),
    "LF20_FVH" : os.path.join(data_dir, "LC22_FVH_220_bpa.tif"),
    "LF22_FVH" : os.path.join(data_dir, "LC22_FVH_230_bpa.tif"),
    "LF22_FDST" : os.path.join(data_dir, "LC22_FDst_230_bpa.tif"),
    "BPS" : os.path.join(data_dir, "LC20_BPS_220_bpa.tif"),
    "ZONE" : os.path.join(data_dir, "us_lf_zones_bpa.tif"),
    "ASPECT" : os.path.join(data_dir, "LC20_Asp_220_bpa.tif"),
    "SLOPE" : os.path.join(data_dir, "LC20_SlpD_220_bpa.tif"),
    "ELEVATION" : os.path.join(data_dir, "LC20_Elev_220_bpa.tif"),
    "BPS_FRG_NE" : os.path.join(data_dir, "BPS_FRG_NEW.tif")

}

models_dir = os.path.join(active_root_dir, "models")
models_fpath_dict = {
    'LF22_F40' : os.path.join(models_dir, "LF22_F40_model_2024-05-29_09-28-44"),
    'LF22_FVT' : os.path.join(models_dir, "LF22_FVT_HGBC_model_2024-05-29_09-37-32"),
    'LF22_FVC' : os.path.join(models_dir, "LF22_FVC_HGBC_model_2024-05-29_09-31-57"),
    'LF22_FVH' : os.path.join(models_dir, "LF22_FVH_HGBC_model_2024-05-29_09-35-38"),
}

out_chunk_dir = os.path.join(active_root_dir, r"outputs\geospatial")
out_raster_dir = os.path.join(data_dir, r'_predicted_rasters')
out_fname_dict = {
    'LF22_F40' : 'Pred_LF22_F40',
    'LF22_FVT' : 'Pred_LF22_FVT',
    'LF22_FVC' : 'Pred_LF22_FVC',
    'LF22_FVH' : 'Pred_LF22_FVH'
}




# Functions

-----

## Make Directory
Creates a directory where data will be output - labeled with the datetime that the script is run. Returns name of directory.

In [14]:
def make_dir(base_dir, file_name):
        """
        Create a directory named using the current datetime.
        Returns:
        - Name of the directory as a string
        """

        datetime = dt.datetime.now().strftime('%Y-%m-%d_%H-%M-%S')
        output_dir = os.path.join(base_dir, file_name + '_' + datetime)

        os.makedirs(output_dir)
        return output_dir

## __Feature Engineering__
Use values from the rasters to create new features. 

### Read Reference Data
Use the LANDFIRE csv tables to create dictionaries to be used to map raster layer values to other features.
1. Separate LF22_FDST into separate features for type, severity, and time since disturbance.
2. Map the BPS value to BPS_NAME in order to reduce cardinality. 

In [15]:
def read_ref_data(ref_data_dir=ref_data_dir):
    """
    Creates dictionaries from LF attributes tables that are used to map layer values to other features. Returns a dictionary of dictionaries. 
    """
    data_dir = ref_data_dir
    BPS_fname = "LF20_BPS_220.csv"

    # Create empty dictionary
    LF_ref_dicts = {}

    # Get BPS reference dictionary
    BPS_df = pd.read_csv(os.path.join(data_dir, BPS_fname))
    LF_ref_dicts["BPS_NAME"] = dict(BPS_df[['VALUE', 'BPS_NAME']].values)

    return LF_ref_dicts                        
        

### Join Features
Use dictionaries created using read_ref_data to map raster values to new features. 

In [16]:
def join_features(df, feature_list = ['BPS_NAME']):
    """
    Joins in additional features using LF attribute tables. 
    """
    
    LF_ref_dicts = read_ref_data()
    
    source_layers = {
        'BPS_NAME' : 'BPS', 
    }

    for feature in feature_list:
        df[feature] = df[source_layers[feature]].map(LF_ref_dicts[feature]).copy()

    return df

## Preprocess window dataframe
Prepares the data to be run through the model. Separates out null and non-burnable values. Returns a dictionary with: 
1. A clean dataframe to be run through the model. 
2. A dataframe of the dropped observations to be rejoined to model predictions. This allows for the data to be reshaped to a 2D numpy array and written as a raster. 

In [17]:
def data_prep(df, target):
    """
    Prepares data to run model on.
    Input: 
    - A dataframe containing created using the flattened numpy arrays returned from reading in the raster chunks. 
    Returns: 
    - Clean dataframe without NULL data or non-burnable F40 Classes 
    - Dataframe with the dropped observations - to be reappended after prediction
    """
    # Based on the target, rename the dropped LF column to Pred column
    column_dict = {
        "LF22_F40" : {'LF20_F40' : 'F40_Pred'},
        "LF22_FVC" : {"LF20_FVC" : "FVC_Pred"},
        "LF22_FVH" : {"LF20_FVH" : "FVH_Pred"},
        "LF22_FVT" : {"LF20_FVT" : "FVT_Pred"}
    }

    pred_dict = {
        "LF22_F40" : "F40_Pred",
        "LF22_FVC" : "FVC_Pred",
        "LF22_FVH" : "FVH_Pred",
        "LF22_FVT" : "FVT_Pred"
    }

    if target != 'LF22_F40':
        df = join_features(df, feature_list = ['BPS_NAME'])

    # Remove -9999/-1111 (Null values)
    null = df[(df.isin([-1111, -9999, -32768])).any(axis=1)]  # Find rows with -1111/-9999 in any column
    df = df.drop(null.index, axis=0)  # Drop those rows

    # Drop non-disturbed values if not predicting F40
    if target != 'LF22_F40':
        non_disturbed = df.loc[df['LF22_FDST'] == 0]
        df = df.drop(non_disturbed.index, axis=0)

    # Filter classes based on the target
    if target == 'LF22_F40':
        # Remove Non-Burnable Classes
        F40_NB = [91, 93]  # Nonburnable F40 Classes
        filtered = df.loc[df['LF20_F40'].isin(F40_NB)]  # Drop NB classes
        df = df.drop(filtered.index, axis=0)
        
    elif target == 'LF22_FVC':
        fvc_filter = list(range(20, 70)) + list(range(80, 86))
        filtered = df.loc[df['LF20_FVC'].isin(fvc_filter)]
        df = df.drop(filtered.index, axis=0)
    
    elif target == 'LF22_FVH':
        fvh_filter = list(range(20, 70)) + list(range(80, 86))
        filtered = df.loc[df['LF20_FVH'].isin(fvh_filter)]
        df = df.drop(filtered.index, axis=0)

    elif target == 'LF22_FVT':
        # Filter out agricultural and developed points
        developed_fvt = list(range(20,33)) + list(range(2901,2906))
        ag_fvt = [80, 81, 82] + list(range(2960, 2971))
        fvt_filter = developed_fvt + ag_fvt
        filtered = df.loc[df['LF20_FVT'].isin(fvt_filter)]
        df = df.drop(filtered.index, axis=0)

    # Join the filtered values together so they can be readded later
    if target == 'LF22_F40':
        dropped = pd.concat([null, filtered], axis=0)
        dropped = dropped.rename(columns=column_dict[target])
    else:
        dropped = pd.concat([null, non_disturbed, filtered], axis=0)
        dropped = dropped.rename(columns=column_dict[target])

    
    return df, dropped[pred_dict[target]]


## Window Read Function
This function is used to read in chunks of the rull raster to be processed. The raster data is stored as blocks with height=1 and width=41855 (the raster width), so we read in chunks composed of these blocks (e.g. 1000 blocks at a time). This corresponds to the row_slice argument. 

In [18]:
def windowed_read(ras, row_slice):
  """
  Reads in a subset (window) of the data to be processed.
  Inputs:
  - ras: Raster object read in using rasterio
  - row_slice: Used to define the window height - in the form (row_start, row_end). 

  Returns:
  - data: The data in the window as a 2D numpy array
  - win: The Window object used to define the subset of the data. 
  - win_transform: The affine transform associated with the window. Used to update the metadata of the output of that chunk. 
  """
  
  with rasterio.open(ras) as src:
    col_slice = (0, src.width)  # Define row slice based on block size
    win = Window.from_slices(row_slice, col_slice) 
    data = src.read(window=win)
    win_transform = src.window_transform(win)
    
  return data, win, win_transform

## __Apply Trained Model__ 
Applies a trained model to the prepared data to predict FVT for each observation (i.e. pixel). After making predictions, rejoins the previously dropped observations so that the dataframe can be reshaped to a 2D numpy array and written to a raster. The index is used to keep track of relative pixel locations in the dataframe. This function returns a dataframe with predictions for each non-null and burnable observation. 

The model was not trained using null (-9999, -1111), non-disturbed, or agricultural/developed classes.

In [19]:
def predict_raster(model, df, target):
    """
    Predicts F40 class given a trained model and data to predict on.
    Returns:
    - A dataframe of predicted F40 values joined to the previously dropped values. 
    """
    # Specify output variable name as function of target
    var_name_dict = {
        "LF22_F40" : "F40_Pred",
        "LF22_FVC" : "FVC_Pred",
        "LF22_FVH" : "FVH_Pred",
        "LF22_FVT" : "FVT_Pred",
    }

    # Prep the data - get the a clean dataframe (i.e. no NULL data/nonburnable classes) and dropped observations
    clean_df, dropped = data_prep(df, target)

    # Get list of predictors for run
    predictors = list(model.feature_names_in_)
    
    X = clean_df[predictors].copy()

    # Run model to predict
    # If clean_df is empty, then all values were NULL and are in dropped
    if clean_df.shape[0] > 0:
        y_pred = model.predict(X)
    else:
        return dropped
    
    # Join the dropped observations back in 
    # This allows the result dataframe to be reshaped back to a raster
    df = pd.DataFrame({var_name_dict[target] : y_pred},
                       index=X.index)
    df = pd.concat([dropped, df])
    df.sort_index(inplace=True)

    # Return predictions
    return df


In [20]:
def window_predict(row, target, model_fpath, win_height, raster_dict, ras_shape, out_dir, out_meta):

    model = load(model_fpath)

    row_start = row
    row_end = row + win_height  # This is also the row_offset of the window

    # make sure slice doesn't exceed row/col dims
    if row_end > ras_shape[0]:
        row_end = ras_shape[0]

    # Define the window to be processed
    row_slice = (row_start, row_end)

    data_dict = {}

    # For the current window, load data from each rasters
    for var, fpath in raster_dict.items():
        data_chunk, data_win, data_transform = windowed_read(fpath, row_slice)
        data_dict[var] = data_chunk.ravel()

    # Create a dataframe from the dictionary of datachunks
    df = pd.DataFrame(data_dict)

    # Look at the window currently processing
    clear_output()

    datetime = dt.datetime.now().strftime('%H-%M-%S-%f')
    out_file = f"data_chunk_{datetime}.tif"
    #print(f"Row Slice: {row_slice}")  
    #print(f"Writing {out_file}.")
    #print(f"Processing window {i} of {floor(ras.shape[0] / win_height)}")

    # Run model to predict F40 Classes for window 
    out_arr = predict_raster(model, df, target)

    # Reshape to 2D
    out_arr_np = out_arr.to_numpy()
    out_arr_2D = out_arr_np.reshape(data_chunk.shape)
    out_arr_2D = out_arr_2D[0]

    # update output metadata for chunk
    out_meta.update({
        'height': out_arr_2D.shape[0],
        'width': out_arr_2D.shape[1],
        'transform' : data_transform
    })

    # Write chunk out
    datetime = dt.datetime.now().strftime('%H-%M-%S-%f')
    out_file = f"data_chunk_{datetime}.tif"
    with rasterio.open(os.path.join(out_dir, out_file), 'w+', **out_meta) as out:
        out.write(out_arr_2D, indexes=1)

In [21]:
def run_model(target, raster_fpaths_dict):
  raster_fpaths = raster_fpaths_dict

  # Define the model to import
  model_fpath = models_fpath_dict[target]

  # Arbitrarily grab metadata from raster to use for updating output metdata
  with rasterio.open(raster_fpaths['LF20_F40']) as src:
    out_meta = src.meta.copy()

  # Open a raster to access its attributes
  ras = rasterio.open(raster_fpaths['LF20_F40'])

  # Create directory using current datetime to output data chunks to
  datetime = dt.datetime.now().strftime('%Y-%m-%d_%H-%M-%S')  # Used to name output file
  out_dir = make_dir(base_dir=out_chunk_dir, file_name=out_fname_dict[target])

  # Define window height and iteration tracker
  win_height = 1000  # Number of rows to process at once

  Parallel(n_jobs=24)(delayed(window_predict)(row, target, model_fpath, win_height, raster_fpaths, ras.shape, out_dir, out_meta) for row in range(0, ras.shape[0], win_height))
  #for row in range(0, ras.shape[0], win_height):
  #   window_predict(row, target, model_fpath, win_height, raster_fpaths, ras.shape, out_dir, out_meta)

  ## Mosaic the data chunks
  # Get the file paths of the generated data chunks
  raster_fpaths = glob.glob(out_dir + "/*.tif")

  # Get the rasterio dataset objects corresponding to each path
  src_files_to_mosaic = []
  for fpath in raster_fpaths:
      src = rasterio.open(fpath)
      src_files_to_mosaic.append(src)

  # Merge the data chunks into a single raster
  mosaic, out_trans = merge(src_files_to_mosaic)

  # Get the metadata for writing
  out_meta = src.meta.copy()
  out_meta.update({
      "driver" : "GTiff",
      "height" : mosaic.shape[1],
      "width" : mosaic.shape[2],
      "transform" : out_trans
  })

  # Write out the mosaic raster

  fname = f"{out_fname_dict[target] + "_" + datetime}.tif"
  with rasterio.open(os.path.join(out_raster_dir, fname), "w", **out_meta) as dest:
      dest.write(mosaic)

  print(f"Raster written to {os.path.join(out_raster_dir, fname)}")
  return os.path.join(out_raster_dir, fname)


# Run Model Predictions
----

In [11]:
pred_fvt_path = run_model('LF22_FVT', raster_fpaths)
raster_fpaths['LF22_FVT'] = pred_fvt_path


targets = ["LF22_FVH", "LF22_FVC"]
pred_fpaths = {}

for target in targets:
    print(f"Processing {target}...")
    pred_fpaths[target] = run_model(target, raster_fpaths)

# Update raster_fpaths using the predicted raster paths
for raster, fpath in pred_fpaths.items():
    raster_fpaths[raster] = fpath

print("Processing F40...")
run_model("LF22_F40", raster_fpaths)

KeyboardInterrupt: 

In [22]:
# Just run the F40 model
print("Processing F40...")
run_model("LF22_F40", raster_fpaths)

KeyboardInterrupt: 

In [58]:
pred_fpaths = {
    'LF22_FVH' : r"\\pnl\projects\BPAWildfire\data\Landfire\fuels_modeling\LF_raster_data\bpa_service_territory\_predicted_rasters\Pred_LF22_FVH_2024-05-29_13-49-26.tif",
    'LF22_FVC' : r"\\pnl\projects\BPAWildfire\data\Landfire\fuels_modeling\LF_raster_data\bpa_service_territory\_predicted_rasters\Pred_LF22_FVC_2024-05-29_14-19-39.tif",
    'LF22_FVT' : r"\\pnl\projects\BPAWildfire\data\Landfire\fuels_modeling\LF_raster_data\bpa_service_territory\_predicted_rasters\Pred_LF22_FVT_2024-05-29_14-59-45.tif"
}

# Update raster_fpaths using the predicted raster paths
for raster, fpath in pred_fpaths.items():
    raster_fpaths[raster] = fpath

print(raster_fpaths)
print("Processing F40...")
run_model("LF22_F40", raster_fpaths)

{'LF20_F40': '\\\\pnl\\projects\\BPAWildfire\\data\\Landfire\\fuels_modeling\\fuelscape_modeling\\..\\LF_raster_data\\bpa_service_territory\\LC22_F40_220_bpa.tif', 'LF20_FVT': '\\\\pnl\\projects\\BPAWildfire\\data\\Landfire\\fuels_modeling\\fuelscape_modeling\\..\\LF_raster_data\\bpa_service_territory\\LC22_FVT_220_bpa.tif', 'LF22_FVT': '\\\\pnl\\projects\\BPAWildfire\\data\\Landfire\\fuels_modeling\\LF_raster_data\\bpa_service_territory\\_predicted_rasters\\Pred_LF22_FVT_2024-05-29_14-59-45.tif', 'LF20_FVC': '\\\\pnl\\projects\\BPAWildfire\\data\\Landfire\\fuels_modeling\\fuelscape_modeling\\..\\LF_raster_data\\bpa_service_territory\\LC22_FVC_220_bpa.tif', 'LF22_FVC': '\\\\pnl\\projects\\BPAWildfire\\data\\Landfire\\fuels_modeling\\LF_raster_data\\bpa_service_territory\\_predicted_rasters\\Pred_LF22_FVC_2024-05-29_14-19-39.tif', 'LF20_FVH': '\\\\pnl\\projects\\BPAWildfire\\data\\Landfire\\fuels_modeling\\fuelscape_modeling\\..\\LF_raster_data\\bpa_service_territory\\LC22_FVH_220_bpa.t

'\\\\pnl\\projects\\BPAWildfire\\data\\Landfire\\fuels_modeling\\fuelscape_modeling\\outputs\\geospatial\\Pred_LF22_F40_2024-05-29_18-41-00\\Pred_LF22_F40_2024-05-29_18-41-00.tif'

In [73]:
for var, path in models_fpath_dict.items():
    model = load(path)
    print(var)
    print(model.feature_names_in_)

LF22_F40
['LF22_FVT' 'LF22_FVH' 'LF22_FVC' 'LF22_FDST' 'ZONE' 'BPS_FRG_NE']


LF22_FVT
['LF20_FVT' 'LF22_FDST' 'ZONE' 'ASPECT' 'SLOPE' 'ELEVATION' 'BPS_NAME'
 'LF20_FVC' 'LF20_FVH']
LF22_FVC
['LF22_FVT' 'LF20_FVC' 'LF22_FDST' 'ZONE' 'BPS_NAME']
LF22_FVH
['LF22_FVT' 'LF20_FVH' 'LF22_FDST' 'ZONE' 'BPS_NAME']
